In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
import keras
from keras.layers import *
from keras.models import Model

In [2]:
training_data_gen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range = 0.12,
    height_shift_range = 0.12,
    width_shift_range = 0.12,
    horizontal_flip = True
)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = training_data_gen.flow_from_directory('./data/images/Images',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='categorical')
test_generator = test_data_gen.flow_from_directory('./data/test',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='categorical')

Found 16666 images belonging to 120 classes.
Found 3911 images belonging to 120 classes.


In [6]:
base_model = InceptionV3(
    weights='imagenet',
    input_shape=(300,300, 3),
    include_top=False)

for layer in base_model.layers:
    layer.trainable = False

In [7]:
inputs = keras.Input(shape=(300, 300, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(120, activation='softmax')(x)

model = Model(base_model.input, predictions)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [8]:
model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [9]:
model.fit(train_generator,
                        epochs=10,
                        validation_data=test_generator)

Epoch 1/10
521/521 [==============================] - 1318s 3s/step - loss: 3.9276 - accuracy: 0.3096 - val_loss: 2.1982 - val_accuracy: 0.7343
Epoch 2/10
521/521 [==============================] - 1272s 2s/step - loss: 1.2191 - accuracy: 0.7908 - val_loss: 0.6039 - val_accuracy: 0.8637
Epoch 3/10
521/521 [==============================] - 1246s 2s/step - loss: 0.6022 - accuracy: 0.8402 - val_loss: 0.4589 - val_accuracy: 0.8729
Epoch 4/10
521/521 [==============================] - 1201s 2s/step - loss: 0.5023 - accuracy: 0.8547 - val_loss: 0.4102 - val_accuracy: 0.8760
Epoch 5/10
521/521 [==============================] - 1180s 2s/step - loss: 0.4540 - accuracy: 0.8613 - val_loss: 0.3777 - val_accuracy: 0.8814
Epoch 6/10
521/521 [==============================] - 1216s 2s/step - loss: 0.4302 - accuracy: 0.8667 - val_loss: 0.3745 - val_accuracy: 0.8862
Epoch 7/10
521/521 [==============================] - 1230s 2s/step - loss: 0.4022 - accuracy: 0.8746 - val_loss: 0.3774 - val_accuracy:

In [10]:
model.save('NN/sanford_dog_inceptionV3.h5')

In [11]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f'\nTest accuracy: {test_acc}')

123/123 - 216s - loss: 0.3655 - accuracy: 0.8829 - 216s/epoch - 2s/step

Test accuracy: 0.8828943967819214
